In [1]:
import dash
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from dash import dcc, html
from dash.dependencies import Input, Output

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "Automobile Sales Statistics Dashboard"

# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

# List of years 
year_list = [i for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    # Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign': 'center', 'color': '#503D36', 'font-size': '24px'}),
    
    # Dropdown to select statistics type
    dcc.Dropdown(
        id='dropdown-statistics',
        options=dropdown_options,
        placeholder='Select a report type',
        value='Yearly Statistics',  # Default value
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    
    # Dropdown to select year (this will be disabled for 'Recession Period Statistics')
    dcc.Dropdown(
        id='select-year',
        options=[{'label': str(i), 'value': i} for i in year_list],
        placeholder='Select Year',
        style={'width': '80%', 'padding': '3px', 'font-size': '20px', 'text-align-last': 'center'}
    ),
    
    # Container for displaying the graphs
    html.Div(id='output-container', children=[])
])


@app.callback(
    [Output('select-year', 'disabled'),
     Output('output-container', 'children')],
    [Input('dropdown-statistics', 'value'),
     Input('select-year', 'value')]
)
def update_output_container(selected_statistics, input_year):
    # Disable year dropdown when 'Recession Period Statistics' is selected, enable otherwise
    if selected_statistics == 'Yearly Statistics':
        year_disabled = False
    else:
        year_disabled = True
    
    # Recession Period Statistics logic
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods where Recession == 1
        recession_data = data[data['Recession'] == 1]

        # Create the charts for Recession Period
        R_chart1 = dcc.Graph(
            figure=px.line(recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index(), 
                           x='Year', y='Automobile_Sales', title="Automobile Sales Fluctuation Over Recession Period")
        )

        R_chart2 = dcc.Graph(
            figure=px.bar(recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                          x='Vehicle_Type', y='Automobile_Sales', title="Average Vehicles Sold by Vehicle Type During Recession")
        )

        R_chart3 = dcc.Graph(
            figure=px.pie(recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                          values='Advertising_Expenditure', names='Vehicle_Type', title="Advertising Expenditure Share by Vehicle Type During Recession")
        )

        R_chart4 = dcc.Graph(
            figure=px.bar(recession_data.groupby(['Vehicle_Type', 'unemployment_rate'])['Automobile_Sales'].mean().reset_index(),
                          x='Vehicle_Type', y='Automobile_Sales', color='unemployment_rate',
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Sales'},
                          title='Effect of Unemployment Rate on Sales During Recession')
        )

        return year_disabled, [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex'})
        ]

    # Yearly Statistics logic
    if selected_statistics == 'Yearly Statistics' and input_year:
        yearly_data = data[data['Year'] == input_year]

        # Create the charts for Yearly Statistics
        Y_chart1 = dcc.Graph(
            figure=px.line(data.groupby('Year')['Automobile_Sales'].mean().reset_index(), 
                           x='Year', y='Automobile_Sales', title="Yearly Automobile Sales")
        )

        Y_chart2 = dcc.Graph(
            figure=px.line(data.groupby('Month')['Automobile_Sales'].sum().reset_index(), 
                           x='Month', y='Automobile_Sales', title="Total Monthly Automobile Sales")
        )

        Y_chart3 = dcc.Graph(
            figure=px.bar(yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index(),
                          x='Vehicle_Type', y='Automobile_Sales', title=f"Average Vehicles Sold by Vehicle Type in {input_year}")
        )

        Y_chart4 = dcc.Graph(
            figure=px.pie(yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index(),
                          values='Advertising_Expenditure', names='Vehicle_Type', title="Total Advertising Expenditure by Vehicle Type")
        )

        return year_disabled, [
            html.Div(className='chart-item', children=[html.Div(children=Y_chart1), html.Div(children=Y_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=Y_chart3), html.Div(children=Y_chart4)], style={'display': 'flex'})
        ]

    return year_disabled, []


# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)

